In [1]:
!pip install pyspark==3.1.2 py4j==0.10.9 

     |████████████████████████████████| 212.4 MB 64 kB/s 
     |████████████████████████████████| 198 kB 55.3 MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.2-py2.py3-none-any.whl size=212880768 sha256=7cb7cf5c1a99e3983c53100130250f988b8afbd3a3db694becb1f5f85f806460
  Stored in directory: /root/.cache/pip/wheels/a5/0a/c1/9561f6fecb759579a7d863dcd846daaa95f598744e71b02c77
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.window import Window
from pyspark.ml.stat import Correlation
from pyspark.sql.types import IntegerType,BooleanType,DateType,FloatType

spark = SparkSession.builder\
        .master("local[*]")\
        .appName('spotify_classification')\
        .getOrCreate()

In [3]:
from google.colab import drive               
drive.mount('/content/drive')

Mounted at /content/drive


# CAMBIARE DATASET

In [29]:
df = spark.read.json("drive/My Drive/spot_data.json")

In [30]:
df.show(5)

+------------+------------------+------------+-----------+------+--------------------+--------------------+----------------+---+--------+--------+----+----------------+-----------+--------------------+---------------------+-------+--------------+-------+
|acousticness|               age|danceability|duration_ms|energy|              genres|            id_track|instrumentalness|key|liveness|loudness|mode|popularity_track|speechiness|sum_artist_followers|sum_artist_popularity|  tempo|time_signature|valence|
+------------+------------------+------------+-----------+------+--------------------+--------------------+----------------+---+--------+--------+----+----------------+-----------+--------------------+---------------------+-------+--------------+-------+
|       0.658|41.794520547945204|       0.602|     156067| 0.552|[classic czech po...|00AeAaSNbe92PRrst...|             0.0|  0|  0.0972|  -6.667|   1|               3|      0.404|               10807|                   80|182.229|    

# Preprocessing

In [31]:
from pyspark.ml.feature import QuantileDiscretizer 

qds = QuantileDiscretizer(relativeError=0.0001, handleInvalid="error", numBuckets=10, inputCol="popularity_track", outputCol="label")

df = qds.setHandleInvalid("keep").fit(df).transform(df)

In [32]:
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(      #con il nuovo dataset aggiungere nuovi attributi/ togliere quelli eliminati per alta correlazione
inputCols=[
 'duration_ms',
 'danceability',
 'energy',
 'key',
 'loudness',
 'mode',
 'speechiness',
 'acousticness',
 'instrumentalness',
 'liveness',
 'valence',
 'tempo',
 'time_signature',
 'sum_artist_followers',
 'sum_artist_popularity'], 
outputCol="feat")
df=assembler.transform(df)

In [33]:
from pyspark.ml.feature import StandardScaler


scaler = StandardScaler(inputCol="feat", outputCol="scaledFeatures")


scalerModel = scaler.fit(df)


df = scalerModel.transform(df)

In [34]:
from pyspark.ml.feature import VarianceThresholdSelector

selector = VarianceThresholdSelector(varianceThreshold=0.1,featuresCol='scaledFeatures', outputCol="features")

df = selector.fit(df).transform(df)

print("Output: Features with variance lower than %f are removed." %
      selector.getVarianceThreshold())

Output: Features with variance lower than 0.100000 are removed.


In [35]:
df.select("features").show(truncate=False)

+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|features                                                                                                                                                                                                                                                                                       |
+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|[1.2361331222982745,3.6367524459446634,2.19643801677356,0.0,-1.3167426258294779,2.108645370323319,2.2425341538160852,1.8900121366

In [36]:
df.show(5)

+------------+------------------+------------+-----------+------+--------------------+--------------------+----------------+---+--------+--------+----+----------------+-----------+--------------------+---------------------+-------+--------------+-------+-----+--------------------+--------------------+--------------------+
|acousticness|               age|danceability|duration_ms|energy|              genres|            id_track|instrumentalness|key|liveness|loudness|mode|popularity_track|speechiness|sum_artist_followers|sum_artist_popularity|  tempo|time_signature|valence|label|                feat|      scaledFeatures|            features|
+------------+------------------+------------+-----------+------+--------------------+--------------------+----------------+---+--------+--------+----+----------------+-----------+--------------------+---------------------+-------+--------------+-------+-----+--------------------+--------------------+--------------------+
|       0.658|41.79452054794

# Scegliere bene treshold e printare le features rimaste (se metto 1 come pasquale mi elimina tutto perchè hanno tutte varianza minore di 1), capire quali sono quelle rimaste

In [37]:
final_data = df.select("id_track", "features", "label")

In [38]:
train, test = final_data.randomSplit([0.7, 0.3], seed = 10)

In [39]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Naive Bayes

In [40]:
from pyspark.ml.classification import NaiveBayes

In [41]:
nb = NaiveBayes()                               # bisogna togliere i valori negativi, credo siano tutti in loudness, come si fa a toglierli senza rifare tutto da capo a parte solo per il nb?
nbModel = nb.fit(train)
predictions = nbModel.transform(test)

Py4JJavaError: ignored

In [ ]:
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction",metricName="accuracy")
evaluator.evaluate(predictions)

In [ ]:
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction",metricName="f1")
evaluator.evaluate(predictions)

In [ ]:
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction",metricName="recallByLabel") #recallbylabel o weightedrecall?
evaluator.evaluate(predictions)

In [ ]:
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction",metricName="precisionByLabel") #precisionbylabel o weightedprecision?
evaluator.evaluate(predictions)

# Random forest

In [42]:
from pyspark.ml.classification import RandomForestClassifier
rf = RandomForestClassifier(featuresCol="features",labelCol="label",maxDepth = 10)
rfModel = rf.fit(train)
predictions = rfModel.transform(test)

In [43]:
rfModel.featureImportances

SparseVector(15, {0: 0.0708, 1: 0.0232, 2: 0.0352, 3: 0.0024, 4: 0.079, 5: 0.0006, 6: 0.0478, 7: 0.1821, 8: 0.0445, 9: 0.0117, 10: 0.0179, 11: 0.0055, 12: 0.0013, 13: 0.2027, 14: 0.2755})

In [44]:
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction",metricName="accuracy")
evaluator.evaluate(predictions)

0.29127608621279505

In [45]:
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction",metricName="f1")
evaluator.evaluate(predictions)

0.24524929815514965

In [46]:
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction",metricName="recallByLabel") 
evaluator.evaluate(predictions)

0.749549333012859

In [47]:
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction",metricName="precisionByLabel") 
evaluator.evaluate(predictions)

0.517786725333112

# MLP

In [48]:
from pyspark.ml.classification import MultilayerPerceptronClassifier

In [49]:
layers = [10, 4, 2, 10] # c'è un modo per scegliere i layers? questi sono scelti a caso

In [50]:
mlp = MultilayerPerceptronClassifier(labelCol='label',
                                            featuresCol='features',
                                            maxIter=100,
                                            layers=layers,
                                            blockSize=128,
                                            seed=1234)

mlpModel = mlp.fit(train)


predictions = mlpModel.transform(test)

In [ ]:
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction",metricName="accuracy")
evaluator.evaluate(predictions)

Py4JJavaError: ignored

In [ ]:
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction",metricName="f1")
evaluator.evaluate(predictions)

In [ ]:
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction",metricName="recallByLabel") 
evaluator.evaluate(predictions)

In [ ]:
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction",metricName="precisionByLabel") 
evaluator.evaluate(predictions)

# LSVM one vs others

In [51]:
from pyspark.ml.classification import LinearSVC, OneVsRest

In [52]:
lsvc = LinearSVC(maxIter=10, regParam=0.1)

In [53]:
ovr = OneVsRest(classifier=lsvc)

In [54]:
ovrModel = ovr.fit(train)

In [55]:
predictions = ovrModel.transform(test)

In [56]:
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction",metricName="accuracy")
evaluator.evaluate(predictions)

0.20261717413616148

In [57]:
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction",metricName="f1")
evaluator.evaluate(predictions)

0.1471408697855798

In [58]:
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction",metricName="recallByLabel") 
evaluator.evaluate(predictions)

0.828746544886432

In [59]:
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction",metricName="precisionByLabel") 
evaluator.evaluate(predictions)

0.18886941279579317

# Logistic regression one vs others

In [60]:
from pyspark.ml.classification import LogisticRegression

lr = LogisticRegression(maxIter=10, tol=1E-6, fitIntercept=True) # i parametri vanno bene?

In [61]:
ovr = OneVsRest(classifier=lr)

In [62]:
ovrModel = ovr.fit(train)

In [63]:
predictions = ovrModel.transform(test)

In [64]:
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction",metricName="accuracy")
evaluator.evaluate(predictions)

0.23162846390694491

In [65]:
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction",metricName="f1")
evaluator.evaluate(predictions)

0.18375327687152626

In [66]:
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction",metricName="recallByLabel") 
evaluator.evaluate(predictions)

0.6367624083643793

In [67]:
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction",metricName="recallByLabel") 
evaluator.evaluate(predictions)

0.6367624083643793